## 언어 모델(LLM)의 두 가지 유형

**LLM**: 문자열을 입력으로 받아 문자열을 반환하는 언어 모델입니다.

**ChatModels**: 메시지 목록을 입력으로 받아 메시지를 반환하는 언어 모델입니다.


---
LLM의 입력/출력은 문자열

하지만 ChatModels은 어떨까요? 입력은 ChatMessage의 목록이고 출력은 하나의 ChatMessage입니다. 


- ChatMessage에는 두 가지 필수 구성 요소:
    - content: 메시지의 내용입니다.
    - role: ChatMessage를 보낸 엔티티의 역할입니다.

---

- **예시**

```python
messages = [{"role": "user", "content": "hi"}]

result = openai.ChatCompletion.create(
    messages=messages,
    model="gpt-3.5-turbo",
    temperature=0
)

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```


---

### LangChain은 서로 다른 역할을 쉽게 구분할 수 있도록 여러 객체를 제공

- HumanMessage: 사람/사용자로부터 오는 ChatMessage입니다.

- AIMessage: 인공지능/어시스턴트가 보내는 ChatMessage.

- SystemMessage: 시스템에서 보내는 챗메시지입니다.

- FunctionMessage: 함수 호출에서 오는 ChatMessage.


이러한 역할 중 어느 것도 적합하지 않은 경우 수동으로 역할을 지정할 수 있는 ChatMessage 클래스도 있습니다.  
이러한 다양한 메시지를 가장 효과적으로 사용하는 방법에 대한 자세한 내용은 프롬프트 가이드를 참조하세요.


predict: 문자열을 받아 문자열을 반환합니다.  
predict_messages: 메시지 목록을 받아 메시지를 반환합니다.  
이러한 다양한 유형의 모델과 다양한 유형의 입력으로 작업하는 방법을 살펴보겠습니다. 먼저 LLM과 ChatModel을 가져와 보겠습니다


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

In [2]:
import openai

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}]
)
print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=[{"role": "user", "content": "what are you doing?"}]
)

print(result["choices"][0]["message"])

{'role': 'assistant', 'content': 'I am an AI language model developed by OpenAI. I am here to answer questions and engage in conversation. How can I assist you today?'}


In [ ]:
# 위의 내용 연습하기

## Chat API : langchain

랭체인에서도 비슷하게 사용할 수 있다.  
좀 더 다양하고 편리한 기능들을 사용할 수 있다.


In [7]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [9]:
# 간단한 템플릿

from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template("나에게 {adjective} {content} 말해줘.")
prompt_template.format(adjective="재밌는", content="농담")

'나에게 재밌는 농담 말해줘.'

In [14]:
message = prompt_template.format(adjective="냉소적인", content="이야기")
message

str

### LangChain은 서로 다른 역할을 쉽게 구분할 수 있도록 여러 객체를 제공

- HumanMessage: 사람/사용자로부터 오는 ChatMessage입니다.

- AIMessage: 인공지능/어시스턴트가 보내는 ChatMessage.

- SystemMessage: 시스템에서 보내는 챗메시지입니다.

- FunctionMessage: 함수 호출에서 오는 ChatMessage.


In [16]:
# 채팅 프롬프트 템플릿

from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = template.format_messages(name="Bob", user_input="What is your name?")

messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, example=False),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, example=False),
 HumanMessage(content='What is your name?', additional_kwargs={}, example=False)]

In [22]:
res = chat(messages)

print(res.content)

content='My name is Bob. How can I assist you today?' additional_kwargs={} example=False


In [ ]:
# 문제: 프롬프트 템플릿을 가지고 여러가지 질문을 만들어보세요

# 1. 공부를 위한 채팅 프롬프트 만들기 질문하기

# 2. 여행계획 세우는 채팅 프롬프트 만들기 질문하기

### Output_parser

딕셔러니 타입으로 출력 가능


In [56]:
review_sample = """
선수: 피에로 인카피에

레버쿠젠 LCB에 인카피에 8카 넣고 100게임 이상 한 후기 남김.

총합 : ★★★☆☆

장점 :
1. 빠른 발(속도 134, 가속 131)
2. 덜 한 역동작과 나쁘지 않은 체감
3. 나름 잘 따주는 공중볼 경합
4. 나쁘지 않은 후방 빌드업(빠른 빌드업 사용 시 수비수들 고립되는 상황이 나오는데 드리블 하면서 몰고 올라갈 수 있음)

단점
1. 몸싸움. 자리를 먼저 잡고 있어도 침투 들어오는 170대 중후반 공격수한테 밀림. 지금 메타가 그렇다고는 하나 든든한 전문 센터백 생각하고 경합 걸면 빡치는 상황 자주 옴.
2. 심각할 정도로 자주 나오는 파울 및 PK. 수비 스탯이 높으면 PK는 잘 안줄줄 알았는데 그 생각이 깨짐. 몸싸움이 강력하지 않아서 공을 확실히 못 뺏어오니까 불필요한 터치가 많아져서 그런가.. 진짜 2~3게임에 한 번은 PK준다고 생각 해야함... 진짜 심각함.
3. 몸싸움이 강력하지 않아 확실한 볼소유를 못해서 발생하는 잦은 아다리 상황. 이것도 진짜 심각함.

총평
19급여를 생각한다면 분명 가성비임이 틀림 없다. 레버쿠젠에 얼마 없는 센터백 자원에서 한 줄기 희망인 것도 사실이다. 그러나 센터백에 요구되는 능력 중 많은 요소가 결격 사유다. 딱 하나 빠른 발만 합격임. 그런데 그 빠른 발이 결국 현 메타에 핵심이기도 함. 19급여 3천억 센터백에 많은걸 바라긴 힘든건 인정.
딱 한 마디로 요약하면 '요긴한 자원이나 안정성이 없다'
"""

review_template = """
다음 텍스트의 경우 다음 정보를 추출합니다.:

name: 선수 이름.

player_Rating: 1~5점 사이로 평가.

summary: 이 플레이어를 한 달락으로 평가하세요. 

다음 키를 사용하여 JSON으로 출력합니다.:
name
player_Rating
summary

text:{text}
"""

In [57]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='\n다음 텍스트의 경우 다음 정보를 추출합니다.:\n\nname: 선수 이름.\n\nplayer_Rating: 1~5점 사이로 평가.\n\nsummary: 이 플레이어를 한 달락으로 평가하세요. \n\n다음 키를 사용하여 JSON으로 출력합니다.:\nname\nplayer_Rating\nsummary\n\ntext:{text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [58]:
messages = prompt_template.format_messages(text=review_sample)
chat = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
response = chat(messages)
print(response.content)

{
  "name": "피에로 인카피에",
  "player_Rating": "3",
  "summary": "요긴한 자원이나 안정성이 없다"
}


In [47]:
# 오류 코드
response.content.get("summery")

AttributeError: 'str' object has no attribute 'get'

In [48]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [74]:
name = ResponseSchema(
    name="name",
    description="선수 이름",
)
player_Rating = ResponseSchema(
    name="player_Rating",
    description="선수의 성능을 1~5점 사이로 평가",
)
summary = ResponseSchema(
    name="summary",
    description="이 플레이어를 한 달락으로 평가하세요.",
)

response_schemas = [name, player_Rating, summary]

In [75]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [76]:
format_instructions = output_parser.get_format_instructions()

format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // 선수 이름\n\t"player_Rating": string  // 선수의 성능을 1~5점 사이로 평가\n\t"summary": string  // 이 플레이어를 한 달락으로 평가하세요.\n}\n```'

In [82]:
review_sample2 = """
선수: 피에로 인카피에

보통 얘를 쓴다는건 레버쿠젠에서 차두리 센터백 파트너 or 왼쪽 풀백으로 쓴다는건데 둘 다 써본 후기 남겨좀
7카 한 150판 정도 썼고 4백 5백 다 써봄

1. CB : 2/5 점
스탯만 보면 전형적인 풀터백임 빠르고 수비스탯 나쁘지않고 키도 어느정도는 커버됨
근데 몸무게가 너무 낮음 이게 경합상황 잦은 센터백에선 너무 치명적임
똑똑하고 빠르고 한데 센터백 특성상 어쩔 수 없이 경합할 수 밖에 없는 상황이 생기는데 그때 너무 무력함
우당탕 골 많이 당하고 뺐었는데도 비비기로 다시 뺐기는 경우 많았음

2. LB 3.5 / 5
무난함 4백 쓸때 풀백으로 쓰면 생각보다 나쁘지않게 사용가능
5백 윙백으로는 비추 공격스탯이 너무 별로고 약발이 너무 치명적임
얘는 걍 안정적으로 주발 패스만 하고 뒤에 수비 안정적으로 하게 두면 밥값잘함
수비는 똑똑하게 잘함 ST, CF 얘들한테나 몸쌈밀리지 윙어들은 경합 이겨먹음

3. 5백 SW : 3.5/ 5
약발이 너무 단점 약발로 잘못 찼다간 패삑 나고 역습당해서 골 먹힘
아예 수비용으로만 쓸거면 나쁘지않음 4점까지 줄 수 있음

결론
차두리 파트너 찾으면 걍 스탯도 비슷한 비커르 쓰셈 얘가 보통 체형에 몸무게도 적당해서 CB은 훨씬 안정적임
나도 첨에 비커르 LB 인카피에 LCB 쓰다가 수비가 너무 털려서 한번 둘이 바꿔보니까 훨씬 나았음
탑소바는 안써봐서 몰루?
"""

review_template = """
다음 텍스트의 경우 다음 정보를 추출합니다.:

name: 선수 이름.

player_Rating: 1~5점 사이로 평가.

summary: 내용을 바탕으로 이 선수에 대한 5문장이하로 요약. 

다음 키를 사용하여 JSON으로 출력합니다.:
name
player_Rating
summary

text:{text}

{format_instructions}
"""

In [83]:
prompt = ChatPromptTemplate.from_template(template=review_template)

messages = prompt.format_messages(
    text=review_sample2, format_instructions=format_instructions
)


print(messages[0].content)


다음 텍스트의 경우 다음 정보를 추출합니다.:

name: 선수 이름.

player_Rating: 1~5점 사이로 평가.

summary: 내용을 바탕으로 이 선수에 대한 5문장이하로 요약. 

다음 키를 사용하여 JSON으로 출력합니다.:
name
player_Rating
summary

text:
선수: 피에로 인카피에

보통 얘를 쓴다는건 레버쿠젠에서 차두리 센터백 파트너 or 왼쪽 풀백으로 쓴다는건데 둘 다 써본 후기 남겨좀
7카 한 150판 정도 썼고 4백 5백 다 써봄

1. CB : 2/5 점
스탯만 보면 전형적인 풀터백임 빠르고 수비스탯 나쁘지않고 키도 어느정도는 커버됨
근데 몸무게가 너무 낮음 이게 경합상황 잦은 센터백에선 너무 치명적임
똑똑하고 빠르고 한데 센터백 특성상 어쩔 수 없이 경합할 수 밖에 없는 상황이 생기는데 그때 너무 무력함
우당탕 골 많이 당하고 뺐었는데도 비비기로 다시 뺐기는 경우 많았음

2. LB 3.5 / 5
무난함 4백 쓸때 풀백으로 쓰면 생각보다 나쁘지않게 사용가능
5백 윙백으로는 비추 공격스탯이 너무 별로고 약발이 너무 치명적임
얘는 걍 안정적으로 주발 패스만 하고 뒤에 수비 안정적으로 하게 두면 밥값잘함
수비는 똑똑하게 잘함 ST, CF 얘들한테나 몸쌈밀리지 윙어들은 경합 이겨먹음

3. 5백 SW : 3.5/ 5
약발이 너무 단점 약발로 잘못 찼다간 패삑 나고 역습당해서 골 먹힘
아예 수비용으로만 쓸거면 나쁘지않음 4점까지 줄 수 있음

결론
차두리 파트너 찾으면 걍 스탯도 비슷한 비커르 쓰셈 얘가 보통 체형에 몸무게도 적당해서 CB은 훨씬 안정적임
나도 첨에 비커르 LB 인카피에 LCB 쓰다가 수비가 너무 털려서 한번 둘이 바꿔보니까 훨씬 나았음
탑소바는 안써봐서 몰루?


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "`

In [84]:
response = chat(messages)

response.content

'```json\n{\n\t"name": "피에로 인카피에",\n\t"player_Rating": "3.5",\n\t"summary": "피에로 인카피에는 센터백으로는 몸무게가 낮아 경합상황에서 무력해지는 단점이 있지만, 똑똑하고 빠르며 수비 스탯이 좋아서 안정적인 선수입니다. 왼쪽 풀백으로 사용할 때도 나쁘지 않으며, 수비용으로만 사용한다면 꽤 좋은 선택입니다."\n}\n```'

In [86]:
output_dict = output_parser.parse(response.content)

output_dict

{'name': '피에로 인카피에',
 'player_Rating': '3.5',
 'summary': '피에로 인카피에는 센터백으로는 몸무게가 낮아 경합상황에서 무력해지는 단점이 있지만, 똑똑하고 빠르며 수비 스탯이 좋아서 안정적인 선수입니다. 왼쪽 풀백으로 사용할 때도 나쁘지 않으며, 수비용으로만 사용한다면 꽤 좋은 선택입니다.'}

In [93]:
for k, v in output_dict.items():
    print(k, v)

name 피에로 인카피에
player_Rating 3.5
summary 피에로 인카피에는 센터백으로는 몸무게가 낮아 경합상황에서 무력해지는 단점이 있지만, 똑똑하고 빠르며 수비 스탯이 좋아서 안정적인 선수입니다. 왼쪽 풀백으로 사용할 때도 나쁘지 않으며, 수비용으로만 사용한다면 꽤 좋은 선택입니다.
